In [5]:
from pymongo import MongoClient
import pandas as pd
from tqdm import tqdm

# Conectar ao MongoDB local
client = MongoClient("mongodb://localhost:27017/")
db = client.fooddb
collection = db.openfoodfacts

# Query: países + idioma em português + product_name não nulo
query = {
    "$or": [
        {"countries_tags": {"$regex": "portugal|brazil", "$options": "i"}},
        {"languages_tags": {"$regex": "pt", "$options": "i"}}
    ],
    "product_name": {"$exists": True, "$ne": ""}
}

In [6]:
# Buscar documentos (limitar campos)
cursor = collection.find(query, {"product_name": 1})

# Converter cursor para lista com tqdm (barra de progresso)
docs = []
for doc in tqdm(cursor, desc="🔄 Convertendo documentos"):
    docs.append(doc)

# Converter para DataFrame
df_filtered = pd.DataFrame(docs)

🔄 Convertendo documentos: 35471it [04:54, 120.26it/s]


In [7]:
# Remover duplicados e entradas inválidas
df_filtered = df_filtered.drop_duplicates(subset="product_name")
df_filtered = df_filtered[df_filtered['product_name'].str.strip() != ""]

# Manter apenas o product_name
df_filtered = df_filtered[["product_name"]]

# Exportar para parquet
df_filtered.to_parquet("produtos_openfoodfacts_pt_br.parquet", index=False)
print(f"💾 Exportado {len(df_filtered)} produtos para produtos_openfoodfacts_pt_br.parquet")

💾 Exportado 29966 produtos para produtos_openfoodfacts_pt_br.parquet
